# Calculate PaleoInclination and PaleoDeclination Grids

This script takes an age grid and plate reconstruction model (presumably self-consistent ones) and derives grids defining the inclination and declination of the magnetising field at each pixel within the oceans. Each pixel is reconstructed bakc to its birth position, based on the rotation file and the age and plate id at that pixel - the plate id is taken from a pre-prepared input grid with consistent dimensions (ie already cookie-cut, since this is currently quite slow). The fixed plate is assumed to be zero, so all pixels are reconstructed back to their absolute location at the time of formation.

In [ ]:
import time
import ocean_remanence_vectors as orv

from gprm.datasets import Reconstructions

#M2019 = Reconstructions.fetch_Muller2019()
M2021 = Reconstructions.fetch_Merdith2021()

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
grd_spacing = 0.1

age_grid_filename = '/Users/simon/Data/GeoData/Rasters/Seafloor_Age_Grid/Seton_etal_2020_PresentDay_AgeGrid.nc'
static_polygon_filename = '/Users/simon/Library/Caches/gprm/Muller2019/Muller_etal_2019_PlateMotionModel_v2.0_Tectonics/StaticGeometries/StaticPolygons/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons_2019_v2.shp'

# Load coregistered grids of seafloor age and plate id.
# If grd_spacing is not specified, we'll use the sampling of the existing age grid
age_grid, plate_id_raster = orv.build_input_grids(age_grid_filename,
                                                  static_polygon_filename,
                                                  grd_spacing)

age_grid.plot()


In [ ]:
# run the reconstruction to get a geodataframe of points on a regular grid,
# with the columns needed to make grids of paleolat and paleodec

start = time.process_time()

(paleo_latitude,
 paleo_declination) = orv.reconstruct_agegrid_to_birthtime(
    M2021, 
    age_grid, 
    plate_id_raster, 
    return_type='xarray')

print(time.process_time() - start)

#rfeatures.plot(column='paleo_inclination')
paleo_latitude.plot()


This section writes out the data to netcdf files. 

[NB GMT doesn't like these grids, even with the 'type' set to NETCDF3, but they read fine in Panoply, matlab, and back into Python]

In [ ]:
paleo_declination.to_netcdf('./grids/paleo_declination_Seton2020_Muller2019_Merdith2021.nc')
paleo_latitude.to_netcdf('./grids/paleo_latitude_Seton2020_Muller2019_Merdith2021.nc')
age_grid.to_netcdf('./grids/paleo_age_Seton2020_Muller2019_Merdith2021.nc')

#ModelName = 'Muller2016_PMAG'

#vt.WriteCDFGrid('../input_grids/%s_PaleoDec_6m.nc' % ModelName,pdagX,pdagY,PaleoDec)
#vt.WriteCDFGrid('../input_grids/%s_PaleoLat_6m.nc' % ModelName,pdagX,pdagY,Yr)
#vt.WriteCDFGrid('../input_grids/%s_AgeGrid_6m.nc' % ModelName,pdagX,pdagY,pdagZ)

